# SQLite Repository Size and Performance
This notebook does some real-world performance testing of the SQLite-backed `Repository`.

This notebook reads all the resources in a cluster and puts them in the `Repository`. We measure various parameters:
* The memory usage -- can it all fit in memory?
* The time to load the data into the `Repository`.
* The time to execute some queries.


## TL;DR

### Data
| Description                                        | Small Cluster | Large Cluster |
|----------------------------------------------------|---------------|---------------|
| Time to download all resources via Kubernetes API  | 10 min 38 s   | 32 min 47 s   |
| Total resouces                                     | 36 766        | 110 948       |
| Resources YAML size                                | 337 MB        | 1.1 GB        |
| Time to load resources from a ZIP                  | ~33.6 s       | 107 s         |
| Measured memory consumption from loading resources | 303.3 MiB     | not measured  |
| Number of `ConfigMap` resources                    | 4438          | 7784          |
| Time to query all `ConfigMap` resources            | 317 ms        | 894 ms        |
| Count of all pairs of `ConfigMap` resources with same name but different namespace | 356 524 | 1 903 728 | 
| Measured memory consumption from querying those pairs of `ConfigMap` resources | 4.3 GiB | <p>OOM Killed initially, then tried retrieving just the first element of each pair:</p><p>14 GiB</p> |
| Time to find all pairs of `ConfigMap` resources `WHERE cm1.name = cm2.name AND cm1.namespace != cm2.namespace` | 22.8 s | <p>OOM killed initially, then tried retrieving just the first element of the pair:</p><p>7 min, 42 s.</p><p>Then reran with `SELECT DISTINCT` to reduce time spent processing results and it ran in:</p><p>7.8 s</p> |
| Time to find all pairs of `ConfigMap` resources `WHERE json_extract(cm1.data,'$.metadata.name') = json_extract(cm2.data,'$.metadata.name') AND json_extract(cm1.data,'$.metadata.namespace') != json_extract(cm2.data,'$.metadata.namespace')` | 199 s | <p>Ran this with `SELECT DISTINCT` and just getting the first element of each pair:</p><p>10 min, 56 s</p> |


### Conclusions
It seems to be able to load large clusters with many resources on them:
- it takes longer to retrieve all the resources from the Kubernetes API than it does to load them in to an
  SQLite-backed `Repository`.
- All resources from even a larger cluster can be loaded to memory

Queries in SQLite are running quickly, though heavy use of `json_extract()` and similar functions will slow things
down a lot. Users can optimize this by specifying constraints on indexed columns like `name` and `namespace`; the
query optimizer will use those indexes before evaluating expressions involving `json_extract()`.

Queries that return a lot of results can be slow or cause the process to get OOM Killed; we should try to protect
ourselves from this.


### Recommendations

1. Implement an object cache in `Repository.Query` to avoid memory overhead of queries that return a set of rows
where the same `Resource` object is returned in multiple rows.
2. Use a decoder/unmarshaler that interns strings to further reduce memory overhead.
3. Implement some limits on the number of results returned.


### Further Research

1. How is SQLite able to evaluate the query using `json_extract()` so quickly (relatively speaking) compared to our
predictions?
2. Why is the measured memory consumption less than the predicted memory consumption?

## Notebook Setup
This notebook requires a Jupyter kernel for Go. We use the Go notebook kernel GoNB, and it is installed
automatically by the `devcontainer.json` file along with the VS Code extensions to support notebooks.

The following script must run, though, to ensure that `kumquat` can be accessed from the notebook.

In [3]:
// Ensure this notebook is able to use the kumquat package from the workspace
!*go mod edit -replace "github.com/guidewire-oss/kumquat=/workspaces/kumquat"
!*cat go.mod

module gonb_5f26382f

go 1.23.1

require github.com/dustin/go-humanize v1.0.1

replace github.com/guidewire-oss/kumquat => /workspaces/kumquat


## Load Resources from Kubernetes Cluster
The following script loads all the resources from the current Kubernetes cluster and saves them as YAML files
in the `resources` directory.

In [28]:
import (
	"context"
	"fmt"
	"os"
	"path/filepath"
	"slices"
	"strings"
	"time"

	k8sjson "k8s.io/apimachinery/pkg/runtime/serializer/json"
	"k8s.io/client-go/dynamic"
	"k8s.io/client-go/tools/clientcmd"
	"k8s.io/client-go/discovery"
	"k8s.io/client-go/restmapper"
	metav1 "k8s.io/apimachinery/pkg/apis/meta/v1"
	"k8s.io/apimachinery/pkg/runtime/schema"
)

func cloneCluster(destinationPath string) {
	start := time.Now()

	// Load the Kubernetes configuration from the default location
	loadingRules := clientcmd.NewDefaultClientConfigLoadingRules()
	configOverrides := &clientcmd.ConfigOverrides{}
	kubeConfig := clientcmd.NewNonInteractiveDeferredLoadingClientConfig(loadingRules, configOverrides)
	config, err := kubeConfig.ClientConfig()
	if err != nil {
		panic(err.Error())
	}

	// Create a dynamic client
	dynamicClient, err := dynamic.NewForConfig(config)
	if err != nil {
		panic(err.Error())
	}

	// Create Kubernetes discovery client
	discoveryClient := discovery.NewDiscoveryClientForConfigOrDie(config)

	// Fetch all available API resources
	preferredResources, err := discoveryClient.ServerPreferredResources()
	if err != nil {
		panic(err.Error())
	}

	// Create a YAML serializer
	yamlSerializer := k8sjson.NewSerializerWithOptions(
		k8sjson.DefaultMetaFactory, nil, nil,
		k8sjson.SerializerOptions{Yaml: true, Pretty: true, Strict: false},
	)

	// Output path
	outputPath, err := filepath.Abs(destinationPath)
	if err != nil {
		panic(err.Error())
	}
	err = os.MkdirAll(outputPath, 0755)
	if err != nil {
		panic(err.Error())
	}

	// Iterate through all available resources
	for _, preferredResource := range preferredResources {
		groupVersion, err := schema.ParseGroupVersion(preferredResource.GroupVersion)
		if err != nil {
			panic(err.Error())
		}
		for _, apiResource := range preferredResource.APIResources {
			if !slices.Contains(apiResource.Verbs, "list") {
				// Skip resources that do not support listing
				continue
			}

			// Define the GroupVersionResource
			gvr := schema.GroupVersionResource{
				Group:    groupVersion.Group,
				Version:  groupVersion.Version,
				Resource: apiResource.Name,
			}

			// Check if the resource is namespaced
			namespace := ""
			if apiResource.Namespaced {
				namespace = metav1.NamespaceAll
			}

			// List all instances of the resource
			resourceList, err := dynamicClient.Resource(gvr).Namespace(namespace).List(context.Background(), metav1.ListOptions{})
			if err != nil {
				fmt.Printf("Error listing resource %s: %v\n", apiResource.Name, err)
				continue
			}

			// Serialize resources to YAML and write to the file
			for _, item := range resourceList.Items {
				var outputFileName string
				if apiResource.Namespaced {
					outputFileName = filepath.Join(outputPath, groupVersion.Group, apiResource.Name, groupVersion.Version, item.GetNamespace(), item.GetName())
				} else {
					outputFileName = filepath.Join(outputPath, groupVersion.Group, apiResource.Name, groupVersion.Version, item.GetName())
				}

				err = os.MkdirAll(filepath.Dir(outputFileName), 0755)
				if err != nil {
					panic(err.Error())
				}

				outputFile, err := os.Create(outputFileName + ".yaml")
				if err != nil {
					panic(err.Error())
				}

				err = yamlSerializer.Encode(&item, outputFile)
				if err != nil {
					fmt.Printf("Failed to write %s: %v\n", item.GetName(), err)
				}

				outputFile.Close()
			}
		}
	}

	duration := time.Since(start)
	fmt.Printf("Cloned cluster in %v\n", duration)
}

%%
cloneCluster("resources")

W0915 15:30:05.137408   33038 warnings.go:70] v1 ComponentStatus is deprecated in v1.19+
W0915 15:36:52.660450   33038 warnings.go:70] ControllerConfig.pkg.crossplane.io/v1alpha1 is deprecated. Use DeploymentRuntimeConfig from pkg.crossplane.io/v1beta1 instead.


Cloned cluster in 10m38.18634s


The results are 36 766 YAML files on disk, taking 337 MB:

In [29]:
!find . | grep yaml | wc -l
!du -ch -d 1 resources

36766
40K	resources/observability.gwcp.io
84K	resources/postgresql.cnpg.io
68K	resources/elbv2.k8s.aws
7.0M	resources/networking.k8s.io
20K	resources/kustomize.toolkit.fluxcd.io
41M	resources/configmaps
6.6M	resources/endpoints
32K	resources/helm.toolkit.fluxcd.io
56K	resources/datadog.jet.crossplane.io
256K	resources/storage.k8s.io
432K	resources/nodes
484K	resources/policy.jspolicy.com
23M	resources/oort.ccs.guidewire.com
6.6M	resources/discovery.k8s.io
43M	resources/secrets
64K	resources/source.toolkit.fluxcd.io
12M	resources/cilium.io
16K	resources/keda.sh
324K	resources/pkg.crossplane.io
544K	resources/coordination.k8s.io
96K	resources/monitoring.kiali.io
4.0M	resources/serviceaccounts
2.8M	resources/acme.cert-manager.io
4.9M	resources/metrics.k8s.io
15M	resources/rbac.authorization.k8s.io
1.7M	resources/namespaces
6.8M	resources/core.oam.dev
340K	resources/persistentvolumeclaims
588K	resources/networking.istio.io
20M	resources/pods
40K	resources/aws.upbound.io
124K	resources/moni

## Repository Loading Performance Test
The following code loads all the YAML files in `resources` into the `Repository`.
The code measures the amount of time it took to load all the files. It also attempts to measure the amount of memory
used by all the resources once they've been loaded.

In [30]:
import (
	"runtime"
	"time"

	"github.com/dustin/go-humanize"
	"github.com/guidewire-oss/kumquat/repository"
	"github.com/magefile/mage/sh"
)

func main() {
	var err error

	err = sh.RunV("ps", "-A", "-l", "-y")
	if err != nil {
		panic(err)
	}

	var m runtime.MemStats

	runtime.ReadMemStats(&m)
	memBefore := m.Sys
	fmt.Printf("Memory Before: %s\n", humanize.IBytes(memBefore))


	var repo repository.Repository
	repo, err = repository.NewSQLiteRepository()
	if err != nil {
		slog.Error("Unable to create repository", "err", err)
		panic(err)
	}
	defer repo.Close()

	start := time.Now()
	err = repository.LoadYAMLFromDirectoryTree(os.DirFS("."), "resources", repo)
	if err != nil {
		slog.Error("Unable to load directory tree", "err", err)
		panic(err)
	}
	duration := time.Since(start)

	fmt.Printf("Loaded resources in %v\n", duration)
	runtime.ReadMemStats(&m)
	memAfter := m.Sys
	fmt.Printf("Memory After: %s\n", humanize.IBytes(memAfter))
	fmt.Printf("Memory Used: %s\n", humanize.IBytes(memAfter-memBefore))

	err = sh.RunV("ps", "-A", "-l", "-y")
	if err != nil {
		panic(err)
	}
}

S   UID     PID    PPID  C PRI  NI   RSS    SZ WCHAN  TTY          TIME CMD
S     0       1       0  0  80   0     0    57 -      ?        00:00:01 docker-init
S     0       7       1  0  80   0  1408   644 -      ?        00:00:02 sh
S     0      26       1  0  80   0 94416 476060 -     ?        00:00:55 dockerd
S     0      38      26  0  80   0 54452 523642 -     ?        00:00:45 containerd
S     0     170       0  0  80   0  1536   644 -      ?        00:00:00 sh
S     0     182     170  0  80   0  1408   644 -      ?        00:00:00 sh
S  1000     191       0  0  80   0  1536   644 pipe_r ?        00:00:00 sh
S     0     283       0  0  80   0  1536   644 -      ?        00:00:00 sh
S  1000     321       0  0  80   0  1664   644 do_wai ?        00:00:00 sh
S  1000     439     321  0  80   0 48340 256008 do_epo ?       00:00:00 node
S  1000     492       0  0  80   0  1536   644 do_wai ?        00:00:00 sh
S  1000     524     492  0  80   0 168448 347209 do_epo ?      00:01:03 nod

"Memory used" may not be trustowrthy, it is only ~18 MiB and we know there's 337 MB on disk. I think the "memory used"
is just reporting on Go heap and not cgo memory allocations for SQLite.

The `ps` command shows that RSS and SZ grew from 26 072 KiB and 407 152 KiB to 298 500 KiB and 507 226 KiB,
respectively. This means that RSS grew 266.0 MiB and SZ grew 97.7 MiB. These values are much more consistent with
the size on disk, though it is weird that they work out to less.

The time to load is reasonable, given we were loading 337 MB of data from a tree of 36 766 files.
If we compress the files and walk the file tree/stream, we can avoid many `fopen()` calls and probably
speed up the loading. Try this:

In [31]:
import (
	"runtime"
	"time"

	"github.com/dustin/go-humanize"
	"github.com/guidewire-oss/kumquat/repository"
	"github.com/magefile/mage/sh"
)

func load(filename string) repository.Repository {
	repo, err := repository.NewSQLiteRepository()
	if err != nil {
		panic(err)
	}

	zipreader, err := zip.OpenReader(filename)
	if err != nil {
		panic(err)
	}
	defer zipreader.Close()

	start := time.Now()
	err = repository.LoadYAMLFromDirectoryTree(zipreader, ".", repo)
	if err != nil {
		panic(err)
	}

	duration := time.Since(start)
	fmt.Printf("Loaded resources in %v\n", duration)

	return repo
}

%%
var err error

err = sh.RunV("ps", "-A", "-l", "-y")
if err != nil {
	panic(err)
}

repo := load("resources-atmos-dev-20240915.zip")
defer repo.Close()

err = sh.RunV("ps", "-A", "-l", "-y")
if err != nil {
	panic(err)
}

S   UID     PID    PPID  C PRI  NI   RSS    SZ WCHAN  TTY          TIME CMD
S     0       1       0  0  80   0     0    57 -      ?        00:00:01 docker-init
S     0       7       1  0  80   0  1408   644 -      ?        00:00:03 sh
S     0      26       1  0  80   0 94416 476060 -     ?        00:00:57 dockerd
S     0      38      26  0  80   0 54452 523642 -     ?        00:00:49 containerd
S     0     170       0  0  80   0  1536   644 -      ?        00:00:00 sh
S     0     182     170  0  80   0  1408   644 -      ?        00:00:00 sh
S  1000     191       0  0  80   0  1536   644 pipe_r ?        00:00:00 sh
S     0     283       0  0  80   0  1536   644 -      ?        00:00:00 sh
S  1000     321       0  0  80   0  1664   644 do_wai ?        00:00:00 sh
S  1000     439     321  0  80   0 48340 256008 do_epo ?       00:00:00 node
S  1000     492       0  0  80   0  1536   644 do_wai ?        00:00:00 sh
S  1000     524     492  0  80   0 167764 347529 do_epo ?      00:01:06 nod

The load time was reduced to 33.6 seconds. This is much more realistic for 36 766 records. It is probably faster than
we could load them through the Kubernetes API. We don't seem to have reached a bottleneck for the `Repository`.

RSS grew from 28 176 KiB to 338 752 KiB, an increase of 303.3 MiB.
SZ grew from 407 104 KiB to 509 269 KiB, an increase of 99.8 MiB.

These sizes are closer to the on-disk footprint of 343 MB.

The next step is to do some queries for performance.


## Analyze Query Performance
Next, we try making some queries on this `Repository`:

In [71]:
%%
repo := load("resources-atmos-dev-20240915.zip")
defer repo.Close()

var m runtime.MemStats

runtime.ReadMemStats(&m)
memBefore := m.Alloc
fmt.Printf("Memory Before: %s\n", humanize.IBytes(memBefore))

start := time.Now()
rs, err := repo.Query(`SELECT cm.data AS cm FROM "ConfigMap.core" AS cm`)
if err != nil {
	panic(err)
}
duration := time.Since(start)

fmt.Printf("Queried %d resources in %v\n", len(rs.Results), duration)
runtime.ReadMemStats(&m)
memAfter := m.Alloc
fmt.Printf("Memory After: %s\n", humanize.IBytes(memAfter))
fmt.Printf("Memory Used: %s\n", humanize.IBytes(memAfter-memBefore))


fmt.Printf("\n\n===== Querying for pairs of ConfigMap (fast: column equality) =====\n")
runtime.ReadMemStats(&m)
memBefore = m.Alloc
fmt.Printf("Memory Before: %s\n", humanize.IBytes(memBefore))
start = time.Now()
rs, err = repo.Query(
	`SELECT cm1.data AS cm1, cm2.data AS cm2 ` +
	`FROM "ConfigMap.core" AS cm1 ` +
	`CROSS JOIN "ConfigMap.core" AS cm2 ` +
	`WHERE cm1.name = cm2.name AND cm1.namespace != cm2.namespace`)
if err != nil {
	panic(err)
}
duration = time.Since(start)

fmt.Printf("Retrieved %d ConfigMap pairs in %v\n", len(rs.Results), duration)
result := rs.Results[0]
fmt.Printf("ConfigMap 1: %s/%s\n", result["cm1"].Namespace(), result["cm1"].Name())
fmt.Printf("ConfigMap 2: %s/%s\n", result["cm2"].Namespace(), result["cm2"].Name())
runtime.ReadMemStats(&m)
memAfter = m.Alloc
fmt.Printf("Memory After: %s\n", humanize.IBytes(memAfter))
fmt.Printf("Memory Used: %s\n", humanize.IBytes(memAfter-memBefore))


fmt.Printf("\n\n===== Querying for pairs of ConfigMap (slow: JSON equality) =====\n")
runtime.ReadMemStats(&m)
memBefore = m.Alloc
fmt.Printf("Memory Before: %s\n", humanize.IBytes(memBefore))
start = time.Now()
rs, err = repo.Query(
	`SELECT cm1.data AS cm1, cm2.data AS cm2 ` +
	`FROM "ConfigMap.core" AS cm1 ` +
	`CROSS JOIN "ConfigMap.core" AS cm2 ` +
	`WHERE json_extract(cm1.data,'$.metadata.name') = json_extract(cm2.data,'$.metadata.name') ` +
	`AND json_extract(cm1.data,'$.metadata.namespace') != json_extract(cm2.data,'$.metadata.namespace')`)
if err != nil {
	panic(err)
}
duration = time.Since(start)

fmt.Printf("Retrieved %d ConfigMap pairs in %v\n", len(rs.Results), duration)
result = rs.Results[0]
fmt.Printf("ConfigMap 1: %s/%s\n", result["cm1"].Namespace(), result["cm1"].Name())
fmt.Printf("ConfigMap 2: %s/%s\n", result["cm2"].Namespace(), result["cm2"].Name())
runtime.ReadMemStats(&m)
memAfter = m.Alloc
fmt.Printf("Memory After: %s\n", humanize.IBytes(memAfter))
fmt.Printf("Memory Used: %s\n", humanize.IBytes(memAfter-memBefore))

Loaded resources in 31.052538s
Memory Before: 32 MiB
Running query query SELECT cm.data AS cm FROM "ConfigMap.core" AS cm
Queried 4438 resources in 317.064ms
Memory After: 72 MiB
Memory Used: 40 MiB


===== Querying for pairs of ConfigMap (fast: column equality) =====
Memory Before: 72 MiB
Running query query SELECT cm1.data AS cm1, cm2.data AS cm2 FROM "ConfigMap.core" AS cm1 CROSS JOIN "ConfigMap.core" AS cm2 WHERE cm1.name = cm2.name AND cm1.namespace != cm2.namespace
Retrieved 356524 ConfigMap pairs in 22.841834s
ConfigMap 1: acolhuan/istio-ca-root-cert
ConfigMap 2: addons/istio-ca-root-cert
Memory After: 4.4 GiB
Memory Used: 4.3 GiB


===== Querying for pairs of ConfigMap (slow: JSON equality) =====
Memory Before: 4.4 GiB
Running query query SELECT cm1.data AS cm1, cm2.data AS cm2 FROM "ConfigMap.core" AS cm1 CROSS JOIN "ConfigMap.core" AS cm2 WHERE json_extract(cm1.data,'$.metadata.name') = json_extract(cm2.data,'$.metadata.name') AND json_extract(cm1.data,'$.metadata.namespace')

There are 4438 `ConfigMap` resources and we can get them all in ~317 ms.

Let's say we want to find all pairs of `ConfigMap` that have the same name (but not self pairs). For this,
we can do a `CROSS JOIN` to join the table with itself.

If we do a `CROSS JOIN`, that's $4438^{2} = 19,695,844$ combinations. If we do comparisons by the `name` and
`namespace` columns, it gets through it in ~23 s.

We also tried a `CROSS JOIN` using `json_extract()` to parse the name and namespace out of the `data` column. In
the worst case, assuming no optimization of the query at all, this is $4438^{2} * 4 = 78,783,376$ JSON parse and
path extract operations. It was done in ~199 s.

Let's see if this is a reasonable time for the evaluation of so many JSON parse operations:

In [59]:
import (
	js "encoding/json"
	"testing"
)

const K8S_OBJECTS = `{"apiVersion":"core.oam.dev/v1beta1","kind":"ComponentDefinition","metadata":{"annotations":{"definition.oam.dev/description":"K8s-objects allow users to specify raw K8s objects in properties","kubectl.kubernetes.io/last-applied-configuration":"{\"apiVersion\":\"core.oam.dev/v1beta1\",\"kind\":\"ComponentDefinition\",\"metadata\":{\"annotations\":{\"definition.oam.dev/description\":\"K8s-objects allow users to specify raw K8s objects in properties\"},\"labels\":{\"gwcp.guidewire.com/app-name\":\"kubevela\",\"gwcp.guidewire.com/created-by\":\"pod-danville\",\"gwcp.guidewire.com/dept\":\"275\",\"gwcp.guidewire.com/maintained-by\":\"pod-danville\",\"gwcp.guidewire.com/tenant-name\":\"quadrant-shared\",\"gwcp.guidewire.com/version\":\"1.9.11\"},\"name\":\"k8s-objects\",\"namespace\":\"vela-system\"},\"spec\":{\"schematic\":{\"cue\":{\"template\":\"output: parameter.objects[0]\\n\\noutputs: {\\n\\tfor i, v in parameter.objects {\\n\\t\\tif i \\u003e 0 {\\n\\t\\t\\t\\\"objects-\\\\(i)\\\": v\\n\\t\\t}\\n\\t}\\n}\\nparameter: objects: [...{}]\\n\"}},\"workload\":{\"type\":\"autodetects.core.oam.dev\"}}}\n","meta.helm.sh/release-name":"vela-core","meta.helm.sh/release-namespace":"vela-system"},"creationTimestamp":"2023-02-27T22:24:36Z","generation":3,"labels":{"app.kubernetes.io/managed-by":"Helm","gwcp.guidewire.com/app-name":"kubevela","gwcp.guidewire.com/created-by":"pod-danville","gwcp.guidewire.com/dept":"275","gwcp.guidewire.com/maintained-by":"pod-danville","gwcp.guidewire.com/tenant-name":"quadrant-shared","gwcp.guidewire.com/version":"1.9.11"},"managedFields":[{"apiVersion":"core.oam.dev/v1beta1","fieldsType":"FieldsV1","fieldsV1":{"f:status":{"f:conditions":{}}},"manager":"794b859c-ab8a-4020-a174-70ea022c2d56","operation":"Update","subresource":"status","time":"2023-06-12T18:43:54Z"},{"apiVersion":"core.oam.dev/v1beta1","fieldsType":"FieldsV1","fieldsV1":{"f:status":{".":{},"f:configMapRef":{},"f:latestRevision":{".":{},"f:name":{},"f:revision":{},"f:revisionHash":{}}}},"manager":"kubevela","operation":"Update","subresource":"status","time":"2023-07-21T12:37:00Z"},{"apiVersion":"core.oam.dev/v1beta1","fieldsType":"FieldsV1","fieldsV1":{"f:metadata":{"f:annotations":{".":{},"f:definition.oam.dev/description":{},"f:kubectl.kubernetes.io/last-applied-configuration":{}},"f:labels":{".":{},"f:gwcp.guidewire.com/app-name":{},"f:gwcp.guidewire.com/created-by":{},"f:gwcp.guidewire.com/dept":{},"f:gwcp.guidewire.com/maintained-by":{},"f:gwcp.guidewire.com/tenant-name":{},"f:gwcp.guidewire.com/version":{}}},"f:spec":{".":{},"f:schematic":{".":{},"f:cue":{".":{},"f:template":{}}},"f:workload":{".":{},"f:type":{}}}},"manager":"HashiCorp","operation":"Update","time":"2024-05-29T23:36:34Z"},{"apiVersion":"core.oam.dev/v1beta1","fieldsType":"FieldsV1","fieldsV1":{"f:metadata":{"f:annotations":{"f:meta.helm.sh/release-name":{},"f:meta.helm.sh/release-namespace":{}}}},"manager":"kubectl-annotate","operation":"Update","time":"2024-06-14T18:01:36Z"},{"apiVersion":"core.oam.dev/v1beta1","fieldsType":"FieldsV1","fieldsV1":{"f:metadata":{"f:labels":{"f:app.kubernetes.io/managed-by":{}}}},"manager":"kubectl-label","operation":"Update","time":"2024-06-14T18:01:37Z"}],"name":"k8s-objects","namespace":"vela-system","resourceVersion":"4395981498","uid":"794b859c-ab8a-4020-a174-70ea022c2d56"},"spec":{"schematic":{"cue":{"template":"output: parameter.objects[0]\n\noutputs: {\n\tfor i, v in parameter.objects {\n\t\tif i > 0 {\n\t\t\t\"objects-\\(i)\": v\n\t\t}\n\t}\n}\nparameter: objects: [...{}]\n"}},"workload":{"definition":{"apiVersion":"","kind":""},"type":"autodetects.core.oam.dev"}},"status":{"conditions":[{"lastTransitionTime":"2023-06-12T18:43:54Z","message":"cannot store capability k8s-objects in ConfigMap: cannot create or update capability k8s-objects in ConfigMap: admission webhook \"mutate-gwcp-atmos-labels-namespace-scoped-resources.guidewire.com\" denied the request: RetrieveObject: Unauthorized","reason":"ReconcileError","status":"False","type":"Synced"}],"configMapRef":"component-schema-k8s-objects","latestRevision":{"name":"k8s-objects-v3","revision":3,"revisionHash":"b2693dc4f89cb830"}}}`

func ParseJsonGo(b []byte) {
	var obj map[string]interface{}
	err := js.Unmarshal(b, &obj)
	if err != nil {
		panic(err)
	}
}

func BenchmarkJsonParse(b *testing.B) {
	data := []byte(K8S_OBJECTS)
	fmt.Printf("Data length: %d\n", len(data))

	for n := 0; n < b.N; n++ {
		ParseJsonGo(data)
	}
}

%test -test.bench=. -test.benchtime=10s

Data length: 4155
goos: linux
goarch: amd64
pkg: gonb_45fd9218
cpu: 06/9e @ 2399.747MHz
BenchmarkJsonParse-8   	Data length: 4155
Data length: 4155
Data length: 4155
  125426	     96187 ns/op
PASS


96.187 µs for each parse of 4155 bytes works out to $\frac{96.187}{10^6} * 4 * 4438^2 = 7577.9$ seconds, which
is way longer than the 199 seconds it took SQLite.

Let's see if SQLite's JSON parsing is any faster:

In [60]:
import (
	js "encoding/json"
	"database/sql"
	"testing"

	_ "github.com/mattn/go-sqlite3"
)

func ParseJsonSQLite(b []byte, db *sql.DB) {
	r := db.QueryRow("SELECT json_extract(?, '$.metadata.name')", b)

	var name string
	err := r.Scan(&name)
	if err != nil {
		panic(err)
	}
	
	// if name != "k8s-objects" {
	// 	panic("Invalid name: " + name)
	// }
}

func BenchmarkJsonParse(b *testing.B) {
	data := []byte(K8S_OBJECTS)
	fmt.Printf("Data length: %d\n", len(data))

	db, err := sql.Open("sqlite3", ":memory:")
	if err != nil {
		panic(err)
	}
	defer db.Close()

	for n := 0; n < b.N; n++ {
		ParseJsonSQLite(data, db)
	}
}

%test -test.bench=. -test.benchtime=10s

Data length: 4155
goos: linux
goarch: amd64
pkg: gonb_45fd9218
cpu: 06/9e @ 2399.747MHz
BenchmarkJsonParse-8   	Data length: 4155
Data length: 4155
Data length: 4155
  168141	     60695 ns/op
PASS


60.695 µs for each parse of 4155 bytes works out to $\frac{60.695}{10^6} * 4 * 4438^2 \approx 4781.8$ seconds, which
is faster that Go's JSON parsing but is still too long to explain how the query executed in ~199 seconds.

So the reason that SQLite is able to evaluate the query so quickly is still a mystery.

---

Going back to the query results, as expected, the memory consumption reported by Go is higher than the loading
experiments, at 4.3 GiB for the `CROSS JOIN` queries. This is because all the 356 524 pairs of `Resource` objects
are on the heap, and that's what's measured. It is high because each `ConfigMap` is instantiated in memory many times.

Given 41 MB of data spread across 4438 `ConfigMap` resources, coming to an average of
$\frac{41,000,000}{4438} \approx 9239$ bytes each, we can approximate the memory footprint of 356 524 resource pairs
with:

$$
\frac{\frac{41,000,000}{4438} * 356524 * 2}{1024^3} \approx 6.1\,GiB
$$

It's interesting that only 4.3 GiB was measured. One would expect the measured value to be higher because the heap
objects are fully parsed as maps of maps; they are not the raw bytes of data. There are a few possibilities:
* Some primitives have more efficient representations in memory than they do as strings
* Some objects like strings might be interned
* The actual distribution of bytes in the `ConfigMap` resources could perhaps result in a smaller overall footprint.
* Differences in the size of YAML-encoded data on disk and JSON-encoded data in the SQLite DB in memory.

Let's check to see if string interning is happening:

In [70]:
import "unsafe"

%%
repo := load("resources-atmos-dev-20240915.zip")
defer repo.Close()

rs, err := repo.Query(
	`SELECT cd.data AS cd FROM "ComponentDefinition.core.oam.dev" AS cd`)
if err != nil {
	panic(err)
}

cd1 := rs.Results[0]["cd"]
cd2 := rs.Results[1]["cd"]

cd1ApiVersion := cd1.Content()["apiVersion"].(string)
cd2ApiVersion := cd2.Content()["apiVersion"].(string)

fmt.Printf("cd1apiVersion: %s\n", cd1ApiVersion)
fmt.Printf("cd2apiVersion: %s\n", cd2ApiVersion)
fmt.Printf("data ptr cd1apiVersion: %p\n", unsafe.StringData(cd1ApiVersion))
fmt.Printf("data ptr cd2apiVersion: %p\n", unsafe.StringData(cd2ApiVersion))

Loaded resources in 31.277349s
Running query query SELECT cd.data AS cd FROM "ComponentDefinition.core.oam.dev" AS cd
cd1apiVersion: core.oam.dev/v1beta1
cd2apiVersion: core.oam.dev/v1beta1
data ptr cd1apiVersion: 0xc0028aa228
data ptr cd2apiVersion: 0xc0028aa360


Since the data pointers are different, we can conclude that string interning isn't happening, and therefore doesn't
explain why the Go representation of the data takes less space than predicted.

It's possible that string interning is happening for the map keys; a decoder might make the assumption that
keys are more likely to be repeated than values. Let's check for interning of the keys:

In [76]:
import "unsafe"

%%
repo := load("resources-atmos-dev-20240915.zip")
defer repo.Close()

rs, err := repo.Query(
	`SELECT cd.data AS cd FROM "ComponentDefinition.core.oam.dev" AS cd`)
if err != nil {
	panic(err)
}

cd1 := rs.Results[0]["cd"].Content()
cd2 := rs.Results[1]["cd"].Content()

for key, _ := range cd1 {
	if key == "apiVersion" {
		fmt.Printf("cd1 apiVersion key ptr is: %p\n", unsafe.StringData(key))
		break;
	}
}

for key, _ := range cd2 {
	if key == "apiVersion" {
		fmt.Printf("cd2 apiVersion key ptr is: %p\n", unsafe.StringData(key))
		break;
	}
}

Loaded resources in 31.953964s
Running query query SELECT cd.data AS cd FROM "ComponentDefinition.core.oam.dev" AS cd
cd1 apiVersion key ptr is: 0xc002010bb0
cd2 apiVersion key ptr is: 0xc002011060


Given the pointers differ, we conclude that there is no string interning happening for keys, either.

The difference between a predicted memory footprint of 6.1 GiB and a measured memory footprint of 4.3 GiB is
still unexplained.


## Analysis of a Bigger Cluster
We should continue this notebook and measure results for some clusters known to be larger.

In [77]:
%%
start := time.Now()
cloneCluster("resources-atmos-mint2-20240915")
duration := time.Since(start)
fmt.Printf("Cloned cluster to disk in %v\n", duration)

W0915 19:38:57.430971   81157 warnings.go:70] v1 ComponentStatus is deprecated in v1.19+
W0915 20:04:30.436965   81157 warnings.go:70] Please use v1beta1 version of SNS group.
W0915 20:04:30.486791   81157 warnings.go:70] Please use v1beta1 version of SNS group.
W0915 20:06:06.128522   81157 warnings.go:70] ControllerConfig.pkg.crossplane.io/v1alpha1 is deprecated. Use DeploymentRuntimeConfig from pkg.crossplane.io/v1beta1 instead.
W0915 20:06:29.773924   81157 warnings.go:70] BucketPolicy has been deprecated. Use spec.forProvider.policy in Bucket instead.


Cloned cluster in 32m47.81822s
Cloned cluster to disk in 32m47.818487s


In [78]:
!find . | grep yaml | wc -l
!du -ch -d 1 resources-atmos-mint2-20240915

110948
256K	resources-atmos-mint2-20240915/elbv2.k8s.aws
142M	resources-atmos-mint2-20240915/networking.k8s.io
20K	resources-atmos-mint2-20240915/kustomize.toolkit.fluxcd.io
89M	resources-atmos-mint2-20240915/configmaps
28M	resources-atmos-mint2-20240915/endpoints
28K	resources-atmos-mint2-20240915/helm.toolkit.fluxcd.io
544K	resources-atmos-mint2-20240915/storage.k8s.io
804K	resources-atmos-mint2-20240915/nodes
488K	resources-atmos-mint2-20240915/policy.jspolicy.com
74M	resources-atmos-mint2-20240915/oort.ccs.guidewire.com
29M	resources-atmos-mint2-20240915/discovery.k8s.io
45M	resources-atmos-mint2-20240915/secrets
13M	resources-atmos-mint2-20240915/cluster.loft.sh
64K	resources-atmos-mint2-20240915/source.toolkit.fluxcd.io
26M	resources-atmos-mint2-20240915/cilium.io
168M	resources-atmos-mint2-20240915/argoproj.io
12M	resources-atmos-mint2-20240915/platform.iac.guidewire.net
24K	resources-atmos-mint2-20240915/resourcequotas
16K	resources-atmos-mint2-20240915/keda.sh
304K	resources-a

110 948 records taking 1.1 GB on disk.

Let's load it and run the same `ConfigMap` queries as before:

In [4]:
%%
repo := load("resources-atmos-mint2-20240915.zip")
defer repo.Close()

var m runtime.MemStats

runtime.ReadMemStats(&m)
memBefore := m.Alloc
fmt.Printf("Memory Before: %s\n", humanize.IBytes(memBefore))

start := time.Now()
rs, err := repo.Query(`SELECT cm.data AS cm FROM "ConfigMap.core" AS cm`)
if err != nil {
	panic(err)
}
duration := time.Since(start)

fmt.Printf("Queried %d resources in %v\n", len(rs.Results), duration)
runtime.ReadMemStats(&m)
memAfter := m.Alloc
fmt.Printf("Memory After: %s\n", humanize.IBytes(memAfter))
fmt.Printf("Memory Used: %s\n", humanize.IBytes(memAfter-memBefore))


fmt.Printf("\n\n===== Querying for pairs of ConfigMap (fast: column equality) =====\n")
runtime.ReadMemStats(&m)
memBefore = m.Alloc
fmt.Printf("Memory Before: %s\n", humanize.IBytes(memBefore))
start = time.Now()
rs, err = repo.Query(
	`SELECT cm1.data AS cm1, cm2.data AS cm2 ` +
	`FROM "ConfigMap.core" AS cm1 ` +
	`CROSS JOIN "ConfigMap.core" AS cm2 ` +
	`WHERE cm1.name = cm2.name AND cm1.namespace != cm2.namespace`)
if err != nil {
	panic(err)
}
duration = time.Since(start)

fmt.Printf("Retrieved %d ConfigMap pairs in %v\n", len(rs.Results), duration)
result := rs.Results[0]
fmt.Printf("ConfigMap 1: %s/%s\n", result["cm1"].Namespace(), result["cm1"].Name())
fmt.Printf("ConfigMap 2: %s/%s\n", result["cm2"].Namespace(), result["cm2"].Name())
runtime.ReadMemStats(&m)
memAfter = m.Alloc
fmt.Printf("Memory After: %s\n", humanize.IBytes(memAfter))
fmt.Printf("Memory Used: %s\n", humanize.IBytes(memAfter-memBefore))

Loaded resources in 1m47.099600162s
Memory Before: 72 MiB
Running query query SELECT cm.data AS cm FROM "ConfigMap.core" AS cm
Queried 7784 resources in 894.377351ms
Memory After: 174 MiB
Memory Used: 103 MiB


===== Querying for pairs of ConfigMap (fast: column equality) =====
Memory Before: 174 MiB
Running query query SELECT cm1.data AS cm1, cm2.data AS cm2 FROM "ConfigMap.core" AS cm1 CROSS JOIN "ConfigMap.core" AS cm2 WHERE cm1.name = cm2.name AND cm1.namespace != cm2.namespace


signal: killed


It runs out of memory.

Even if I increase Rancher Desktop memory from 16 GiB to 24 GiB, it runs out of memory.

Let's try to cut memory consumption in half by only returning one half of each pair:

In [5]:
%%
repo := load("resources-atmos-mint2-20240915.zip")
defer repo.Close()

var m runtime.MemStats

runtime.ReadMemStats(&m)
memBefore := m.Alloc
fmt.Printf("Memory Before: %s\n", humanize.IBytes(memBefore))

start := time.Now()
rs, err := repo.Query(`SELECT cm.data AS cm FROM "ConfigMap.core" AS cm`)
if err != nil {
	panic(err)
}
duration := time.Since(start)

fmt.Printf("Queried %d resources in %v\n", len(rs.Results), duration)
runtime.ReadMemStats(&m)
memAfter := m.Alloc
fmt.Printf("Memory After: %s\n", humanize.IBytes(memAfter))
fmt.Printf("Memory Used: %s\n", humanize.IBytes(memAfter-memBefore))


fmt.Printf("\n\n===== Querying for pairs of ConfigMap (fast: column equality) =====\n")
runtime.ReadMemStats(&m)
memBefore = m.Alloc
fmt.Printf("Memory Before: %s\n", humanize.IBytes(memBefore))
start = time.Now()
rs, err = repo.Query(
	`SELECT cm1.data AS cm1 ` +
	`FROM "ConfigMap.core" AS cm1 ` +
	`CROSS JOIN "ConfigMap.core" AS cm2 ` +
	`WHERE cm1.name = cm2.name AND cm1.namespace != cm2.namespace`)
if err != nil {
	panic(err)
}
duration = time.Since(start)

fmt.Printf("Retrieved %d ConfigMap pairs in %v\n", len(rs.Results), duration)
result := rs.Results[0]
fmt.Printf("ConfigMap 1: %s/%s\n", result["cm1"].Namespace(), result["cm1"].Name())
runtime.ReadMemStats(&m)
memAfter = m.Alloc
fmt.Printf("Memory After: %s\n", humanize.IBytes(memAfter))
fmt.Printf("Memory Used: %s\n", humanize.IBytes(memAfter-memBefore))

Loaded resources in 1m43.290806535s
Memory Before: 94 MiB
Running query query SELECT cm.data AS cm FROM "ConfigMap.core" AS cm
Queried 7784 resources in 1.381678273s
Memory After: 185 MiB
Memory Used: 91 MiB


===== Querying for pairs of ConfigMap (fast: column equality) =====
Memory Before: 185 MiB
Running query query SELECT cm1.data AS cm1 FROM "ConfigMap.core" AS cm1 CROSS JOIN "ConfigMap.core" AS cm2 WHERE cm1.name = cm2.name AND cm1.namespace != cm2.namespace
Retrieved 1903728 ConfigMap pairs in 7m42.00921249s
ConfigMap 1: acme-mgt37dr-prod/istio-ca-root-cert
Memory After: 14 GiB
Memory Used: 14 GiB


The number of `ConfigMap` pairs is vastly increased: from 356 524 pairs in the old cluster to 1 903 728 pairs in
this cluster. The memory scales linearly with the number of pairs, and the memory requirement should have been
$\frac{4.3\,GiB}{356524}*1903728 \approx 22.96\,GiB$. We only needed half the memory (because we only retrieved half of each
pair) and so our memory requirement was only 11.5 GiB, and we measured 14 GiB which mostly agrees.

We should probably implement an object cache for the `Resource` objects returned by `Repository.Query` so that we can
avoid huge memory usage. This is added to the recommendations section at the bottom of this notebook.

We can remove the Go memory and CPU overhead by using `SELECT DISTINCT` to de-duplicate the results in
the DB itself:

In [6]:
%%
repo := load("resources-atmos-mint2-20240915.zip")
defer repo.Close()

var m runtime.MemStats

fmt.Printf("\n\n===== Querying for pairs of ConfigMap (fast: column equality) =====\n")
runtime.ReadMemStats(&m)
memBefore := m.Alloc
fmt.Printf("Memory Before: %s\n", humanize.IBytes(memBefore))
start := time.Now()
rs, err := repo.Query(
	`SELECT DISTINCT cm1.data AS cm1 ` +
	`FROM "ConfigMap.core" AS cm1 ` +
	`CROSS JOIN "ConfigMap.core" AS cm2 ` +
	`WHERE cm1.name = cm2.name AND cm1.namespace != cm2.namespace`)
if err != nil {
	panic(err)
}
duration := time.Since(start)

fmt.Printf("Retrieved %d ConfigMap pairs in %v\n", len(rs.Results), duration)
result := rs.Results[0]
fmt.Printf("ConfigMap 1: %s/%s\n", result["cm1"].Namespace(), result["cm1"].Name())
runtime.ReadMemStats(&m)
memAfter := m.Alloc
fmt.Printf("Memory After: %s\n", humanize.IBytes(memAfter))
fmt.Printf("Memory Used: %s\n", humanize.IBytes(memAfter-memBefore))

Loaded resources in 1m52.8397515s


===== Querying for pairs of ConfigMap (fast: column equality) =====
Memory Before: 111 MiB
Running query query SELECT DISTINCT cm1.data AS cm1 FROM "ConfigMap.core" AS cm1 CROSS JOIN "ConfigMap.core" AS cm2 WHERE cm1.name = cm2.name AND cm1.namespace != cm2.namespace
Retrieved 3587 ConfigMap pairs in 7.762753973s
ConfigMap 1: acme-mgt37dr-prod/istio-ca-root-cert
Memory After: 78 MiB
Memory Used: 16 EiB


SQLite is blazingly fast at this! 7.8 seconds to get everything is amazing.

This makes us wonder how quickly it will execute `json_extract()` when the decoding overhead is removed.

In the previous data set, there were 4438 `ConfigMap` resources and the query took ~199 s. Now there are 7784
`ConfigMap` resources. The number of comparisons should scale quadratically with the number of `ConfigMap` resources,
so we predict the time it will take:

$$
\frac{199 \,s}{4438^2} * 7784^2 \approx 612\,s
$$

which is 10 minutes and 12 seconds. Let's do the query:

In [7]:
%%
repo := load("resources-atmos-mint2-20240915.zip")
defer repo.Close()

var m runtime.MemStats

fmt.Printf("\n\n===== Querying for pairs of ConfigMap (fast: column equality) =====\n")
runtime.ReadMemStats(&m)
memBefore := m.Alloc
fmt.Printf("Memory Before: %s\n", humanize.IBytes(memBefore))
start := time.Now()
rs, err := repo.Query(
	`SELECT DISTINCT cm1.data AS cm1 ` +
	`FROM "ConfigMap.core" AS cm1 ` +
	`CROSS JOIN "ConfigMap.core" AS cm2 ` +
	`WHERE json_extract(cm1.data,'$.metadata.name') = json_extract(cm2.data,'$.metadata.name') ` +
	`AND json_extract(cm1.data,'$.metadata.namespace') != json_extract(cm2.data,'$.metadata.namespace')`)
if err != nil {
	panic(err)
}
duration := time.Since(start)

fmt.Printf("Retrieved %d ConfigMap pairs in %v\n", len(rs.Results), duration)
result := rs.Results[0]
fmt.Printf("ConfigMap 1: %s/%s\n", result["cm1"].Namespace(), result["cm1"].Name())
runtime.ReadMemStats(&m)
memAfter := m.Alloc
fmt.Printf("Memory After: %s\n", humanize.IBytes(memAfter))
fmt.Printf("Memory Used: %s\n", humanize.IBytes(memAfter-memBefore))

Loaded resources in 1m39.448198097s


===== Querying for pairs of ConfigMap (fast: column equality) =====
Memory Before: 89 MiB
Running query query SELECT DISTINCT cm1.data AS cm1 FROM "ConfigMap.core" AS cm1 CROSS JOIN "ConfigMap.core" AS cm2 WHERE json_extract(cm1.data,'$.metadata.name') = json_extract(cm2.data,'$.metadata.name') AND json_extract(cm1.data,'$.metadata.namespace') != json_extract(cm2.data,'$.metadata.namespace')
Retrieved 3587 ConfigMap pairs in 10m56.194448502s
ConfigMap 1: acme-mgt37dr-prod/istio-ca-root-cert
Memory After: 71 MiB
Memory Used: 16 EiB


The predicted time was 10 minutes and 12 seconds; the actual time was 10 minutes and 56 seconds.

An interesting experiment is to see if the query optimizer will be smart enough to avoid `json_extract()` in cases
where the column equality constraints have already failed. To test, let's add

```sql
AND cm1.name = cm2.name AND cm1.namespace != cm2.namespace
```

to the `WHERE` clause that uses `json_extract()`:

In [10]:
%%
repo := load("resources-atmos-mint2-20240915.zip")
defer repo.Close()

fmt.Printf("\n\n===== Querying for pairs of ConfigMap (JSON equality and column equality) =====\n")
start := time.Now()
rs, err := repo.Query(
	`SELECT DISTINCT cm1.data AS cm1 ` +
	`FROM "ConfigMap.core" AS cm1 ` +
	`CROSS JOIN "ConfigMap.core" AS cm2 ` +
	`WHERE json_extract(cm1.data,'$.metadata.name') = json_extract(cm2.data,'$.metadata.name') ` +
	`AND json_extract(cm1.data,'$.metadata.namespace') != json_extract(cm2.data,'$.metadata.namespace') ` +
	`AND cm1.name = cm2.name AND cm1.namespace != cm2.namespace`)
if err != nil {
	panic(err)
}
duration := time.Since(start)

fmt.Printf("Retrieved %d ConfigMap pairs in %v\n", len(rs.Results), duration)
result := rs.Results[0]
fmt.Printf("ConfigMap 1: %s/%s\n", result["cm1"].Namespace(), result["cm1"].Name())

Loaded resources in 1m37.931242669s


===== Querying for pairs of ConfigMap (JSON equality and column equality) =====
Running query query SELECT DISTINCT cm1.data AS cm1 FROM "ConfigMap.core" AS cm1 CROSS JOIN "ConfigMap.core" AS cm2 WHERE json_extract(cm1.data,'$.metadata.name') = json_extract(cm2.data,'$.metadata.name') AND json_extract(cm1.data,'$.metadata.namespace') != json_extract(cm2.data,'$.metadata.namespace') AND cm1.name = cm2.name AND cm1.namespace != cm2.namespace
Retrieved 3587 ConfigMap pairs in 21.033986849s
ConfigMap 1: acme-mgt37dr-prod/istio-ca-root-cert


At 21 seconds, it's slower than using columns directly (7.8 seconds!) but faster than using only `json_extract()`,
which took 10 minutes and 56 seconds. So there is some basic query optimization being performed.